In [4]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
import numpy as np
import random
#https://simplemaps.com/data/us-cities

In [ ]:

def read_large_csv(file_path, chunk_size):
    """Reads a large CSV file in chunks and returns a generator of chunks."""
    return pd.read_csv(file_path, chunksize=chunk_size, low_memory=False)

def process_large_csv(file_path, chunk_size=100000):
    """Processes the large CSV file and returns the cleaned DataFrame."""
    df_combined = pd.DataFrame()

    for chunk in read_large_csv(file_path, chunk_size):
        df_combined = pd.concat([df_combined, chunk], ignore_index=True)

    return df_combined


def drop_columns(data_frame):
    cols_to_drop = ['ID', 'Source', 'End_Lat', 'End_Lng', 'Distance(mi)', 
                    'Description', 'Airport_Code', 'Pressure(in)', 
                    'Wind_Direction', 'Precipitation(in)', 
                    'Amenity','Give_Way', 'No_Exit', 'Traffic_Calming', 
                    'Turning_Loop', 'Nautical_Twilight', 'Astronomical_Twilight',
                    'Civil_Twilight','Airport_Code','Sunrise_Sunset'
                    ]
    return data_frame.drop(cols_to_drop, axis=1)


def split_and_save_by_state(data_frame, output_dir=r"..\data\state_wise_data"):
    """Splits data by state and saves it into separate CSV files."""
    os.makedirs(output_dir, exist_ok=True)

    for state, state_df in data_frame.groupby("State"):
        if (state == 'FL' or state == 'CA' or state == 'VA' or state == 'NY') :
            state_file = os.path.join(output_dir, f"{state}_org.csv")
            state_df.to_csv(state_file, index=False)
    print(f"Data saved for FA, CA, VA, NY \n ")

def save_cleaned_all_state_data(data_frame, output_dir=r"..\data\cleaned_raw_data"):
    os.makedirs(output_dir, exist_ok=True)
    filePath = os.path.join(output_dir, "cleaned_all_state_data.csv")

    data_frame.to_csv(filePath, index=False)
    print(f"Saved Cleaned all state Data  \n ")


def print_summary(data_frame):
    print("** Data Frame Info: \n")
    print(f"{data_frame.info()}")
    print("\n ** Data Frame description: \n")
    print(data_frame.describe(include="all"))
    print("\n The shape of data is:",(all_state_raw_data_file_frame.shape))
    print("\n Top 3 records \n")
    display(all_state_raw_data_file_frame.head(3))
    print(" \n Unique Source: ", all_state_raw_data_file_frame['Source'].unique())

def plot_histogram_state_vs_accident_count(data_frame):
    """Analyzes the state-wise accident count and plots a histogram."""
    state_counts = data_frame["State"].value_counts()
    
    print(f"\nNumber of unique states: {state_counts.count()}")

    # Plot histogram for state vs accident count
    plt.figure(figsize=(12, 6))
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))   
    state_counts.plot(kind="bar", color="skyblue", edgecolor="black")
    plt.xlabel("State")
    plt.ylabel("Total Accidents")
    plt.title("Total Accidents by State")
    plt.xticks(rotation=90)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()


def plot_histogram_weather_condition_vs_accident_count(data_frame):
    """Analyzes the state-wise accident count and plots a histogram."""
    weather_condition = data_frame["Weather_Condition"].value_counts()
    
    plt.figure(figsize=(25, 20))
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))   
    weather_condition.plot(kind="bar", color="skyblue", edgecolor="black")
    plt.xlabel("Weather")
    plt.ylabel("Total Accidents")
    plt.title("Total Accidents by weather condition")
    plt.xticks(rotation=90)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

def plot_us_map_accident_count(data_frame):
    state_counts = data_frame["State"].value_counts().reset_index()
    state_counts.columns = ["State", "Accidents"]

    fig = px.choropleth(
        state_counts, 
        locations="State", 
        locationmode="USA-states", 
        color="Accidents",
        color_continuous_scale="reds",
        title="Number of US Accidents for each State"
    )
    fig.update_layout(geo=dict(scope="usa"))
    fig.show()

def plot_histogram_street_vs_accident_count(data_frame):
    """Analyzes the state-wise accident count and plots a histogram."""
    weather_condition = data_frame["Street"].value_counts()
    
    plt.figure(figsize=(25, 20))
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))   
    weather_condition.plot(kind="bar", color="skyblue", edgecolor="black")
    plt.xlabel("Street")
    plt.ylabel("Total Accidents")
    plt.title("Total Accidents by weather condition")
    plt.xticks(rotation=90)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

def plot_histogram_top_20_citi_accident_count(data_frame, text=""):

    # Get the top 20 cities with the highest number of accidents
    top_cities = data_frame["City"].value_counts().sort_values()[-20:].reset_index()
    top_cities.columns = ["city", "number_of_accidents"]

    # Plot the bar chart
    plt.figure(figsize=(20,7))
    sns.barplot(x="city", y="number_of_accidents", hue="city", data=top_cities, palette='Set2', legend=False)  # ✅ FIXED
    plt.title(f"TOP 20 CITIES WITH HIGHEST NUMBER OF ACCIDENTS - {text}", fontsize=20)
    plt.xticks(rotation=40)
    plt.show()



In [9]:
all_state_raw_data_file_frame = process_large_csv(raw_file_path, chunk_size)

In [10]:
print("Raw Data Summary. \n")
print_summary(all_state_raw_data_file_frame)


Raw Data Summary. 

** Data Frame Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day


 
 Unique Source:  ['Source2' 'Source3' 'Source1']
